In [1]:
import datajoint as dj
import numpy as np

DataJoint 0.4.1 (Oct 28, 2016)
Loading local settings from environment variables
Connecting edgera@localhost:3306


In [2]:
#dj.conn()

In [3]:
schema = dj.schema('ex1', locals())

In [4]:
@schema
class Experiment(dj.Manual):
    definition = """
    ex1.Experiment (manual)     # A generic experiment.
    experiment_id : int
    ----
    """

    
@schema
class DataCollection(dj.Manual):
    definition = """
    # A way to group datasets per experiment
    -> ex1.Experiment
    datacollection_id : int     # Unique identifier for the dataset.
    -----
    """
 

@schema
class RawData(dj.Manual):
    definition = """
    # Raw collected data.
    -> ex1.DataCollection
    data_id : int           # A unique data instance id.
    -----
    x : float
    y : float 
    """
  
    
@schema
class LinearModel(dj.Computed):
    definition = """
    # fits line a DataCollection. y=mx+b form
    -> ex1.DataCollection
    -----
    m : float     # Slope
    b : float     # intercept
    """
    @property
    def populated_from(self):
        print('ey yo')
        return RawData
    
    def _make_tuples(self, key):
        rel =  RawData() & key
        data = rel.fetch()  
        
        X = np.array([tup[3] for tup in data])
        Y = np.array([tup[4] for tup in data])
        
        A = np.vstack([X, np.ones(len(X))]).T
        m, b = np.linalg.lstsq(A, Y)[0]
        
        
        tup = key
        tup['m'] = m
        tup['b'] = b
        self.insert1(tup)
    
    
@schema
class MSE(dj.Computed):
    definition = """
    # Computes Mean Square Error for a particular Model
    -> ex1.LinearModel
    -----
    val : float         # The MSE value.
    """
    @property
    def populated_from(self):
        return LinearModel
    
    def _make_tuples(self, key):
        rel =  RawData() & key
        data = rel.fetch()  
        
        X = np.array([tup[3] for tup in data])
        Y = np.array([tup[4] for tup in data])
        
        rel = LinearModel() & key
        model = rel.fetch1()
        
        yCalc = X*model['m'] + model['b']
        mse = ((Y - yCalc) ** 2).mean(axis=0)

        tup = key
        tup['val'] = mse
        self.insert1(tup)

    
@schema
class R2(dj.Computed):
    definition = """
    # Computes Mean Square Error for a particular Model
    -> ex1.LinearModel
    -----
    val : float         # The MSE value.
    """
    @property
    def populated_from(self):
        return LinearModel
    
    def _make_tuples(self, key):
        rel =  RawData() & key
        data = rel.fetch()  
        
        X = np.array([tup[3] for tup in data])
        Y = np.array([tup[4] for tup in data])
        
        rel = LinearModel() & key
        model = rel.fetch1()
        
        yCalc = X*model['m'] + model['b']
        r2  = 1-np.sum((Y - yCalc)**2)/np.sum((Y - np.mean(Y))**2)
       
        tup = key
        tup['val'] = r2
        self.insert1(tup)
        
#schema.spawn_missing_classes()

In [5]:
# Generate some data
Experiment().insert1(dict(experiment_id=1))
DataCollection().insert1([1,1])

n_data = 10
X = np.random.randint(10, size=n_data)
Y = np.zeros(n_data)
mu = 0
sigma = .1

for i in range(n_data):
    Y[i] = 2 * X[i] + np.random.normal(mu, sigma)
    RawData().insert1([1, 1, i+1, X[i], Y[i]])

LinearModel().populate()
MSE().populate()
R2().populate()


In [6]:
Experiment().delete()

The contents of the following tables are about to be deleted:
`ex1`.`experiment` (1 tuples)
`ex1`.`data_collection` (1 tuples)
`ex1`.`__linear_model` (1 tuples)
`ex1`.`__m_s_e` (1 tuples)
`ex1`.`__r2` (1 tuples)
`ex1`.`raw_data` (10 tuples)
Proceed? [yes, No]: yes
Done
